In [1]:
####
#Author: yuehao wang
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
#import awscli
import selenium
import boto3
import pandas as pd
import time
import s3fs

from selenium import webdriver


####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome("D:\dev\chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)

<selenium.webdriver.remote.webelement.WebElement (session="3b52973f113672c1f13fbe85e420cd83", element="277c15c2-f56b-4aae-a561-6c1944f4eb46")>


In [2]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
6,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
7,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
8,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
9,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY


### Step 6. Note that the output of the file has an issue! The header inserts a blank row! Update the script that was provided to remove the blank row in the csv output file


### I use the df.to_csv(index=False) to remove the blank header.

In [3]:
def save_to_s3(df):
    
    # df.to_csv(None, index=False) can remove the blank header
    byte_encoded_csv = df.to_csv(None, index=False).encode() #encodes file as binary
    
    # s3://g2-final/readme.txt
    pathname = 's3://yw-m10-database-update-bucket/' #specify location of s3:/{my-bucket}/
    filename= 'yuehao_wang_m10_' #name of your group
    datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
    
    filenames3 = "%s%s%s.csv"%(pathname, filename, datetime) #name of the filepath and csv file
    print(filenames3)
    
    s3 = s3fs.S3FileSystem(anon=False)
    
    with s3.open(filenames3, 'wb') as file:
        file.write(byte_encoded_csv) #writes byte-encoded file to s3 location

    #print success message
    print("Successfull uploaded file to location:"+str(filenames3))
    
    return filenames3


s3_filename = save_to_s3(df)

s3://yw-m10-database-update-bucket/yuehao_wang_m10_20210403000043.csv
Successfull uploaded file to location:s3://yw-m10-database-update-bucket/yuehao_wang_m10_20210403000043.csv


### Step 7. Test the file by inspecting the output on the s3 bucket

In [13]:
# test the csv. download from s3, than print it
test_df = pd.read_csv(str(s3_filename))
test_df.head(2)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,NaN,NaN,NaN,NaN,NaN,NaN
1,"""Studio 5404"" Inc.",44-39-58,463180470.0,NFP,MASSAPAQUA,NY


### save csv file to local

In [9]:
save_path = "C:/Users/amorn/Documents/yeshiva/workspace/M10_Assignment/files/"+str(s3_filename[35:])
test_df.to_csv(save_path)

### The example of having the blank header

In [10]:
# This file has the blank header
test_df = pd.read_csv('s3://yw-m10-database-update-bucket/yuehao_wang_m10_20210402232617.csv')
test_df.head(2)

,Unnamed: 0,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"""Studio 5404"" Inc.",44-39-58,463180470.0,NFP,MASSAPAQUA,NY
